# Regression Assumptions & Logistic Regression Notebook

1. Outlier Detection and Removal
2. Multicollinearity 
3. Scaling 
4. Logistic Regression

### Import the libraries you need!


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
pd.set_option("display.max_columns",None) 
pd.set_option("display.max_rows",None) 

import warnings
warnings.filterwarnings("ignore")

from IPython.display import Image
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn.neighbors import LocalOutlierFactor
sns.set(style="darkgrid", palette="pastel", color_codes=True)
sns.set_context('talk')

from pathlib import Path
data_dir = Path('../input/images')

### Read the dataset

In [ ]:
df_1 = pd.read_csv("../input/heart-disease-uci/heart.csv")
df_1.head()

#### 1. Show the scatter plot between chol and target

In [ ]:
df_1.plot.scatter(x = 'chol', y = 'target')

<h1>I. Outlier Removal</h1>

This section is adapted from Suraj RP's excellent notebook on various outlier detection techniques. Visit his full notebook here: https://www.kaggle.com/rpsuraj/outlier-detection-techniques-simplified

## Exploratory Data Analysis

### Painting a picture of the data!

#### 0. What columns do we have in this dataset?

In [ ]:
df_1.columns

#### 1. How many null values do we have in each column?

In [ ]:
df_1.isnull().sum()

#### 2. What is the distribution in each column?

In [ ]:
df_1.hist()

#### 3. Check out the statistical summary of the dataframe.

In [ ]:
df_1.describe()

#### Let's consider serum cholestoral in mg/dl column i.e. "chol" for our analysis.

#### 4. Plot a simple box plot to visualize the outliers.

In [ ]:
df_1.boxplot('chol')

#### From the above box plot, we can surely observe that there are outliers in it!

## Back to Preprocessing...

#### 1. Find the value at the 25th percentile. 

In [ ]:
p25 = df_1.chol.quantile(0.25)

#### 2. Find the value at the 75th percentile.

In [ ]:
p75 = df_1.chol.quantile(0.75)

#### 3. Find the IQR. 

In [ ]:
iqr = p75 - p25

#### 4. Calculate the cut-off range.

In [ ]:
cutoff = 1.5*iqr

#### 5. Calculate the upper bound value using the cut-off range and the 75th percentile value.

In [ ]:
upper = p75+cutoff

#### 6. Calculate the lower bound value using the cut-off range and the 25th percentile value.

In [ ]:
lower = p25 - cutoff

#### 7. What is the total number of outliers?

In [ ]:
df_1[(df_1.chol > upper) | (df_1.chol < lower)].count()

### Visual representation:

In [ ]:
plt.figure(figsize = (10,6))
sns.distplot(df_1.chol, kde=False)
plt.axvspan(xmin = lower,xmax= df_1.chol.min(),alpha=0.2, color='red')
plt.axvspan(xmin = upper,xmax= df_1.chol.max(),alpha=0.2, color='red')

#### Here the red zone represents the outlier zone! The records present in that zone are considered as outliers

### Remedial Measure:
#### 8. Remove the records which are above the upper bound value and records below the lower bound value!

In [ ]:
#Data Frame without outliers

df_1.drop(df_1[df_1.chol > upper].index, inplace = True)

# II. Multicollinearity 

### Back to Exploratory Data Analysis

#### 1. Find the correlation coefficients matrix.

In [ ]:
arr = df_1.to_numpy()
arr = np.matrix.transpose(arr)
coef_mat = np.corrcoef(arr)


#### 2. Plot the correlation matrix as a heatmap.

In [ ]:
sns.heatmap(coef_mat)

### How to deal with multicollinearity?

1. Using Principal Component Analysis to select the most important variables
2. Remove one of the columns that contribute to multicollinearity
3. Create interaction variables from the correlated variables

# III. Scaling

#### 1. Display the minimum and maximum of each column.

In [ ]:
print("max:\n"+str(df_1.max()))
print()
print("min:\n"+str(df_1.min()))

#### 2 Find the max of 'chol'

In [ ]:
df_1.chol.max()

#### 3. Find the min of 'chol'

In [ ]:
df_1.chol.min()

#### 4. Perform min-max scaling on 'chol'

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cholarr = df_1.chol.to_numpy().reshape(-1,1)
scaler.fit(cholarr)
print(scaler.data_min_)
print(scaler.data_max_)
scaler.transform(cholarr)

#### 5. Find the mean of 'age'

In [ ]:
age_mean = df_1.age.mean()

#### 6. Find the standard deviation of 'age'

In [ ]:
age_std = df_1.age.std()

#### 7. Verify the mean and standard deviation by showing the distribution of 'age.'

In [ ]:
df_1.hist("age")

#### 8. Perform Z-standardization on 'age.'

In [ ]:
agestd = (df_1.age - age_mean)/age_std

# IV. Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
import scipy

#### 1. Create a logistic regression model

In [ ]:
model = LogisticRegression()

#### 2. Set X to be the cholestrol column and Y to be the target column.

In [ ]:
x = df_1.chol
y = df_1.target

#### 3. Perform train-test-splot by 20%

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

#### 4. Fit the model to the train data.

In [ ]:
model.fit(x_train.to_numpy().reshape(-1,1), y_train.to_numpy().reshape(-1,1))

In [ ]:
print(model.intercept_, model.coef_)

In [ ]:
def plot_log_reg(x, y, data, clf, xmin=None, xmax=None, alpha=1, ax=None):
    plt.rcParams["figure.figsize"] = [10,10]
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.figure
    ax.scatter(data[x], data[y], color='black', zorder=20, alpha=alpha)
    if xmin is None:
        xmin = x.min()
    if xmax is None:
        xmax = x.max()
    X_test = np.linspace(xmin, xmax, 300)

    loss = scipy.special.expit(X_test * clf.coef_ + clf.intercept_).ravel()
    ax.plot(X_test, loss, linewidth=3)

    ax.set_xlabel(x)
    ax.set_ylabel(y)
    fig.tight_layout()
    sns.despine()
    return fig, ax

In [ ]:
plot_log_reg('chol', 'target', df_1[['chol','target']], clf = model, xmin = 100, xmax = 350);
